Output Parser

In [9]:
import json

from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_core.output_parsers import CommaSeparatedListOutputParser, JsonOutputParser
from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions} return only the list.",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = OllamaLLM(model="llama3.1")

chain = prompt | llm | output_parser

result = chain.invoke({"subject": "programming languages"})

result[:5]

['Python', 'Java', 'JavaScript', 'C++', 'Ruby']

In [14]:
class CuisineRecipe(BaseModel):
    name: str = Field(description="The name of the recipe.")
    recipe: str = Field(description="The recipe instructions.")

output_parser = JsonOutputParser(pydantic_object=CuisineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "The name of the recipe.", "title": "Name", "type": "string"}, "recipe": {"description": "The recipe instructions.", "title": "Recipe", "type": "string"}}, "required": ["name", "recipe"]}
```


In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n return only the JSON object.",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "The name of the recipe.", "title": "Name", "type": "string"}, "recipe": {"description": "The recipe instructions.", "title": "Recipe", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n return only the JSON object.'


In [17]:
chain = prompt | llm | output_parser

result = chain.invoke(
    {
        "query": "Give me a recipe for a simple Italian pasta dish with tomato sauce and basil."
    })

print(result)

{'name': 'Spaghetti Aglio e Olio', 'recipe': 'Cook spaghetti according to package instructions. Heat olive oil in a pan over medium heat. Add minced garlic and cook until fragrant. Stir in chopped fresh basil. Serve with cooked pasta.'}
{"name": "Spaghetti Aglio e Olio", "recipe": "Cook spaghetti according to package instructions. Heat olive oil in a pan over medium heat. Add minced garlic and cook until fragrant. Stir in chopped fresh basil. Serve with cooked pasta."}
